In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal,beta,dirichlet, norm
from math import log
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)
from scipy.special import gamma
import ternary


np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

In [119]:
# a1 = 50
# a2 = 50
# a3 = 50
# beta = gamma(a1)*gamma(a2)*gamma(a3)/gamma(a1+a2+a3)

In [78]:
sz = 10
x1 = norm.rvs(-2,1,size=sz)
x2 = norm.rvs(2,1,size=sz)
x = np.hstack((x1,x2))

In [98]:
c_id = np.zeros(x.shape)
clusters,cluster_counts = np.unique(c_id,return_counts=True)
n_cluster = len(clusters)

sigma0 = 1
mu0 = 0

# sigma_cluster = np.array([sigma0])
# mu_cluster = np.array([mu0])

sigma_y = 1
tau_y = 1/(sigma_y**2)
ite = 30
alpha = 10

In [99]:
# rand_x = p0.rvs(size=2*sz)

# rand_x_prob = dirichlet.pdf(rand_x,size=2*sz)

# pred_x = norm.rvs(rand_x,sigma0,size=2*sz) * rand_x_prob

In [100]:
for _ in range(ite):
    for i in range(sz*2):
        
        assign_id = int(c_id[i])
        cluster_counts[assign_id] -= 1
        if cluster_counts[assign_id] == 0:
            cluster_counts[assign_id] = cluster_counts[-1]
            c_id[np.where(c_id==n_cluster-1)[0]] = assign_id
            clusters_counts = np.delete(cluster_counts, n_cluster-1) 
            n_cluster -= 1
        c_id[i] = -1
        # print(assign_id)
        # print(cluster_counts)

        logp = np.zeros((n_cluster+1))
        for j in range(n_cluster):

            idx = np.where(c_id == j)[0] # element indices within clusters[j]
            y_average = np.mean(x[idx])
            prior_tau_j = 1/(sigma0**2)
            mu_j = (y_average*cluster_counts[j]* tau_y + mu0 * prior_tau_j )/ (cluster_counts[j]* tau_y + prior_tau_j )
            sigma_j = np.sqrt(1/(cluster_counts[j]* tau_y + prior_tau_j ) + sigma_y**2)
            
            logp[j] = np.log(cluster_counts[j]/(sz*2-1+alpha)) + norm.logpdf(x[i],mu_j,sigma_j)
        new_cluster_mu = mu0
        new_cluster_sigma = np.sqrt(sigma0**2+sigma_y**2)
        
        logp[-1] = np.log(alpha/(sz*2-1+alpha)) + norm.logpdf(x[i],new_cluster_mu,new_cluster_sigma)
        print(logp)
        logp_max = np.max(logp)
        logp = logp - logp_max
        p = np.exp(logp)
        p = p/np.sum(p)
        
        new_c = np.random.choice(np.arange(n_cluster+1),p=p)
        if new_c == n_cluster:
            # clusters = np.append(clusters,new_c)
            n_cluster += 1
            cluster_counts = np.append(cluster_counts,1)
        else:
            cluster_counts[new_c] += 1
        c_id[i] = new_c
        # print(cluster_counts)






[-1.958 -2.901]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [97]:
print(cluster_counts)

[3.000 2.000 1.000 10.000 6.000 2.000 1.000 1.000 1.000 1.000 2.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1

In [ ]:
# start = 0
# scale = 10
# for i in range(start, scale + (1 - start)):
#     for j in range(start, scale + (1 - start) - i):
#         k = scale - i - j
#         print(i, j, k)

In [121]:
# # showcasing Dirichlet distribution
# def f(x):
#     return x[0]**(a1-1) * x[1]**(a2-1) * x[2]**(a3-1)/beta

# scale = 60
# fig, tax = ternary.figure(scale=scale)
# tax.heatmapf(f,scale=scale)

In [ ]:
# def shannon_entropy(p):
#     """Computes the Shannon Entropy at a distribution in the simplex."""
#     s = 0.
#     for i in range(len(p)):
#         try:
#             s += p[i] * log(p[i])
#         except ValueError:
#             continue
#     return -1.*s
# scale = 60

# figure, tax = ternary.figure(scale=scale)
# tax.heatmapf(shannon_entropy, boundary=True, style="triangular")
# tax.boundary(linewidth=2.0)
# tax.set_title("Shannon Entropy Heatmap")

# tax.show()